In [1]:
## Data scrapping prpm
from PyClasses.Scrapers.PRPMScraper import PRPMScraper
PRPMscrap = PRPMScraper()

## find sinonim still buggy (will work if got time)
result = PRPMscrap.findWordMetaData('selain')
print(result)

[<b>Bersinonim dengan </b>]
{'selain': {'meanings': ['1. hanya, kecuali: ia tidak dapat berbuat apa-apa ~ membiarkan sahaja; 2. di samping, bersama-sama dgn: ~ ceramah, diadakan juga pembacaan sajak;', ', = lain drpd 1 di samping: ~ berlatih kawad, mereka belajar undang-undang juga. 2 kecuali; hanya: Emaknya tidak dpt berbuat apa-apa, ~ menangis.'], 'synonym': ['berbeza', 'tidak sama;']}}


In [2]:
from datetime import datetime, timedelta
from PyClasses.Scrapers.BHarianScraper import BHarianScraper

timeNow = datetime.now()
bharianScraper = BHarianScraper()
fromDate = datetime(2024,12,4)
print(f'From Date: {fromDate}')
articlesData = bharianScraper.scrapArticles(fromDate=fromDate,category='/sukan/bola')
timeEnd = datetime.now()
timeTaken = timeEnd - timeNow
print(f'Processing Start Time: {timeNow}')
print(f'Processing End Time: {timeEnd}')
print(f'Total Time Taken: {timeTaken}')
#print(bharianScraper.scrapOneArticle('/sukan/bola/2024/12/1332562/hamidin-lepas-jawatan-presiden-fam'))

From Date: 2024-12-04 00:00:00
Article teaser found on page 0.
Article teaser found on page 1.
Article teaser found on page 2.
Article teaser found on page 3.
Scrapping 61 article links ...
1. Scrap success on url: '/sukan/bola/2024/12/1333701/banjir-tak-halang-alif-pamer-prestasi-terbaik'
2. Scrap success on url: '/sukan/bola/2024/12/1333699/penggantungan-pfa-tak-jejas-program-pembangunan-bola-sepak-perlis'
3. Scrap success on url: '/sukan/bola/2024/12/1333667/kinoshi-sambut-kemenangan-pertama-dalam-kesedihan'
4. Scrap success on url: '/sukan/bola/2024/12/1333640/sentuhan-magis-salah-jadi-taruhan-liverpool'
5. Scrap success on url: '/sukan/bola/2024/12/1333558/vicente-mahu-tinggalkan-kenangan-manis'
6. Scrap success on url: '/sukan/bola/2024/12/1333547/aksi-piala-asean-hadiah-buat-vicente-syamer'
7. Scrap success on url: '/sukan/bola/2024/12/1333529/neuer-digantung-dua-aksi-susulan-kad-merah-pertama'
8. Scrap success on url: '/sukan/bola/2024/12/1333510/pemain-ecuador-diselamatkan-dar

In [3]:
for data in articlesData:
    print(data)

{'Title': 'Banjir tak halang Alif pamer prestasi terbaik', 'authDetail': {'authName': 'Malik Muhamad', 'relLink': '/authors/malik-muhamad', 'email': 'bhsukan@bh.com.my'}, 'publishTime': 'Disember 6, 2024 @ 5:29pm', 'articleBrief': 'Muhammad Alif (tengah) ketika berdepan KDN FC pada Rabu lalu. Foto TFC', 'contents': ['KUALA TERENGGANU: Biarpun rumah dilanda banjir gelombang kedua, pertahanan Terengganu FC (TFC), Muhammad Alif Zakaria mengetepikan seketika ujian itu demi tugasan bersama pasukannya itu.', 'Hasilnya pada aksi liga Super berdepan pasukan jiran, Kelantan Darul Naim (KDN) FC pada Rabu lalu, pemain berusia 26 tahun itu berjaya membawa impak yang besar buat pasukan.', 'Bukan sahaja jaringan pada babak pertamanya yang boleh dikatakan berkelas dunia, tetapi lebih penting bek kiri itu berjaya membantu kemenangan 2-1 pasukannya ke atas lawan.', '"Alhamdulillah rezeki saya selepas berjaya membantu pasukan melakar kemenangan keenam pada musim ini dalam saingan Liga Super.', '"Kami ti

In [ ]:
print (bharianScraper.relLinkToArticle_all[-1])